In [2]:
from typing import List, Dict, Iterable, Tuple, Callable
from matplotlib import pyplot as plt
from collections import Counter
# import pygal
import sys
import os
import string
import urllib.request
import requests
import curl
import pycurl
import webbrowser
import numpy as np
import math
import pandas as pd
# from IPython import qt
from matplotlib.pyplot import figure
from py.xml import raw
from requests.api import get
from matplotlib import pyplot as plt
# from scratch.working_with_data import rescale
# from scratch.multiple_regression import least_squares_fit, predict
# from scratch.gradient_descent import gradient_step

# from stats import mean, median, de_mean, standard_deviation, correlation
# from gradient_descent import minimize_stochastic, maximize_stochastic, maximize_batch
# from vector import dot, vector_add
# from normal import normal_cdf
# from matrix import make_matrix, get_column, shape, matrix_multiply
# from logistic_regression import *

import math
import os
import random
import sys
from functools import partial, reduce

from scipy.optimize import fmin_tnc

import tqdm

from typing import*

from collections import*
# from scipy import*
from sklearn.metrics import*

from numpy import *

# import mnist
# bltin_sum = np.sum

In [3]:
# Functions

# def add(a, b): return a + b

Vector = List[float]

Tensor = list

def vector_sum(vectors):
    """Sums all corresponding elements"""
    # Check that vectors is not empty
    assert vectors, "no vectors provided!"

    # Check the vectors are all the same size
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), "different sizes!"

    # the i-th element of the result is the sum of every vector[i]
    return [sum(vector[i] for vector in vectors)
            for i in range(num_elements)]

def scalar_multiply(c , v):
    """Multiplies every element by c"""
    return [c * v_i for v_i in v]

def vector_mean(vectors):
    """Computes the element-wise average"""
    n = len(vectors)
    m = sum(vectors,axis=0)
    vec_mean = np.multiply(1/n,m)
    return vec_mean

def de_mean(xs):
    """Translate xs by subtracting its mean (so the result has mean 0)"""
    x_bar = np.mean(xs)
    d_mean = [x - x_bar for x in xs]
    return d_mean

def dot(v, w):
    """Computes v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w), "vectors must be same length"

#     return np.sum(v_i * w_i for v_i, w_i in zip(v, w))
#     gen = 
    return np.sum(np.fromiter((v_i * w_i for v_i, w_i in zip(v, w)),float))

def sum_of_squares(v):
    """Returns v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

def variance(xs):
    """Almost the average squared deviation from the mean"""
    assert len(xs) >= 2, "variance requires at least two elements"

    n = len(xs)
    deviations = de_mean(xs)
    vari = sum_of_squares(deviations)/(n-1)
    return vari

# Standard deviation                        
def standard_deviation(xs):
    """The standard deviation is the square root of the variance"""
    std_dev = np.sqrt(variance(xs)) 
    return std_dev

def scale(data):
    """returns the mean and standard deviation for each position"""
    dim = data.shape[0]
    
    # Vector Mean
#     n = len(data)
#     m = np.sum(data,axis=0)
#     means = np.multiply(1/n,m)
    means = vector_mean(data)
    
    # Standard Deviaiton
    stdevs = [standard_deviation([vector[i] for vector in data])
              for i in range(dim)]
    return means,stdevs

def rescale(data):
    """
    Rescales the input data so that each position has
    mean 0 and standard deviation 1. (Leaves a position
    as is if its standard deviation is 0.)
    """
    dim = data.shape[0]
    means, stdevs = scale(data)
    
    means = list(means)
    stdevs = list(stdevs)

    # Make a copy of each vector
    rescaled = [v[:] for v in data]
    v0 = []
    for v in rescaled:
        v = list(v)
        for i in range(dim):
            if stdevs[i] > 0:
                v[i] = (v[i] - means[i]) / stdevs[i]
        v0.append(v)

    return v0

def gradient_step(v, gradient, step_size):
    """Moves `step_size` in the `gradient` direction from `v`"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    grad_step = np.add(v,step)
    return grad_step

# def predict(alpha, beta, x_i):
#     pred = beta * x_i + alpha
#     return pred

# def error(x, y, beta):
#     """
#     The error from predicting beta * x_i + alpha
#     when the actual value is y_i
#     """
#     err_fin = predict(alpha, beta, x_i) - y_i
#     return err_fin

def predict(x, beta):
    """assumes that the first element of x is 1"""
    return dot(x, beta)

def error(x, y, beta):
    return predict(x, beta) - y 

def sqerror_gradient(x, y, beta):
    err = error(x, y, beta)
    err_fin = [2 * err * x_i for x_i in x]
    return err_fin

def least_squares_fit(xs, ys, learning_rate = 0.001, num_steps = 1000, batch_size = 1):
    """
    Find the beta that minimizes the sum of squared errors
    assuming the model y = dot(x, beta).
    """
    # Start with a random guess
    guess = [np.random.random() for _ in xs[0]]

    for _ in tqdm.trange(num_steps, desc="least squares fit"):
        for start in range(0, len(xs), batch_size):
            batch_xs = xs[start:start+batch_size]
            batch_ys = ys[start:start+batch_size]

            gradient = vector_mean([sqerror_gradient(x, y, guess)
                                    for x, y in zip(batch_xs, batch_ys)])
            guess = gradient_step(guess, gradient, -learning_rate)

    return guess

def logistic(x):
    return 1.0 / (1 + math.exp(-x))

def logistic_prime(x):
    y = logistic(x)
    return y * (1 - y)

def _negative_log_likelihood(x, y, beta):
    """The negative log likelihood for one data point""" 
    if y == 1:
        return -math.log(logistic(dot(x, beta)))
    else:
        return -math.log(1 - logistic(dot(x, beta)))
    
def negative_log_likelihood(xs, ys, beta):
    return sum(_negative_log_likelihood(x, y, beta)
               for x, y in zip(xs, ys))

def _negative_log_partial_j(x, y, beta, j):
    """
    The jth partial derivative for one data point.
    Here i is the index of the data point.
    """
    return -(y - logistic(dot(x, beta))) * x[j]

def _negative_log_gradient(x, y, beta):
    """
    The gradient for one data point.
    """
    return [_negative_log_partial_j(x, y, beta, j)
            for j in range(len(beta))]

def negative_log_gradient(xs, ys,beta):
    return vector_sum([_negative_log_gradient(x, y, beta)
                       for x, y in zip(xs, ys)])

def split_data(data, prob):
    """Split data into fractions [prob, 1 - prob]"""
    data = data[:]                    # Make a shallow copy
    random.shuffle(data)              # because shuffle modifies the list.
    cut = int(len(data) * prob)       # Use prob to find a cutoff
    return data[:cut], data[cut:]     # and split the shuffled list there.

def train_test_split(xs, ys, test_pct):
     # Generate the indices and split them
    idxs = [i for i in range(len(xs))]
    train_idxs, test_idxs = split_data(idxs, 1 - test_pct)

    return ([xs[i] for i in train_idxs],  # x_train 
            [xs[i] for i in test_idxs],   # x_test
            [ys[i] for i in train_idxs],  # y_train
            [ys[i] for i in test_idxs])   # y_test
                                                                
def step_function(x: float) -> float:
    return 1.0 if x >= 0 else 0.0

def sigmoid(t: float) -> float: 
    return 1 / (1 + math.exp(-t))

# Gradient Descent - step
def gradient_step(v: Vector, gradient: Vector, step_size: float):
    """Moves `step_size` in the `gradient` direction from `v`"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    return add(v, step)

def sum_of_squares_gradient(v: Vector) -> Vector:
    return [2 * v_i for v_i in v]

def squared_distance(v: Vector, w: Vector) -> float:
    """Computes (v_1 - w_1) ** 2 + ... + (v_n - w_n) ** 2"""
    return sum_of_squares(subtract(v, w))

def distance(v: Vector, w: Vector) -> float:
    """Computes the distance between v and w"""
    return math.sqrt(squared_distance(v, w))

def shape(tensor: Tensor) -> List[int]:
    sizes: List[int] = []
    while isinstance(tensor, list):
        sizes.append(len(tensor))
        tensor = tensor[0]
    return sizes

def is_1d(tensor: Tensor) -> bool:
    """
    If tensor[0] is a list, it's a higher-order tensor.
    Otherwise, tensor is 1-dimensional (that is, a vector).
    """
    return not isinstance(tensor[0], list)

def tensor_sum(tensor: Tensor) -> float:
    """Sums up all the values in the tensor"""
    if is_1d(tensor):
        return sum(tensor)  # just a list of floats, use Python sum
    else:
        return sum(tensor_sum(tensor_i)      # Call tensor_sum on each row
                   for tensor_i in tensor)   # and sum up those results.

def tensor_apply(f: Callable[[float], float], tensor: Tensor) -> Tensor:
    """Applies f elementwise""" 
    if is_1d(tensor):
        return [f(x) for x in tensor]
    else:
        return [tensor_apply(f, tensor_i) for tensor_i in tensor] 
    
def zeros_like(tensor: Tensor) -> Tensor:
    return tensor_apply(lambda _: 0.0, tensor)


def tensor_combine(f: Callable[[float, float], float],
                   t1: Tensor,
                   t2: Tensor) -> Tensor:
    """Applies f to corresponding elements of t1 and t2"""
    if is_1d(t1):
        return [f(x, y) for x, y in zip(t1, t2)]
    else:
        return [tensor_combine(f, t1_i, t2_i)
                for t1_i, t2_i in zip(t1, t2)]
    
def inverse_normal_cdf(p: float,
                       mu: float = 0,
                       sigma: float = 1,
                       tolerance: float = 0.00001) -> float:
    """Find approximate inverse using binary search"""

    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)

    low_z = -10.0                      # normal_cdf(-10) is (very close to) 0
    hi_z  =  10.0                      # normal_cdf(10)  is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # Consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the CDF's value there
        if mid_p < p:
            low_z = mid_z              # Midpoint too low, search above it
        else:
            hi_z = mid_z               # Midpoint too high, search below it

    return mid_z

def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def softmax(tensor: Tensor) -> Tensor:
    """Softmax along the last dimension"""
    if is_1d(tensor):
        # Subtract largest value for numerical stability.
        largest = max(tensor)
        exps = [math.exp(x - largest) for x in tensor]

        sum_of_exps = sum(exps)                 # This is the total "weight."
        return [exp_i / sum_of_exps             # Probability is the fraction
                for exp_i in exps]              # of the total weight.
    else:
        return [softmax(tensor_i) for tensor_i in tensor]

In [4]:

class Layer:
    """
    Our neural networks will be composed of Layers, each of which
    knows how to do some computation on its inputs in the "forward"
    direction and propagate gradients in the "backward" direction.
    """
    def forward(self, input):
        """
        Note the lack of types. We're not going to be prescriptive
        about what kinds of inputs layers can take and what kinds
        of outputs they can return.
        """
        raise NotImplementedError

    def backward(self, gradient):
        """
        Similarly, we're not going to be prescriptive about what the
        gradient looks like. It's up to you the user to make sure
        that you're doing things sensibly.
        """ 
        raise NotImplementedError

    def params(self) -> Iterable[Tensor]:
        """
        Returns the parameters of this layer. The default implementation
        returns nothing, so that if you have a layer with no parameters
        you don't have to implement this.
        """
        return ()

    def grads(self) -> Iterable[Tensor]:
        """
        Returns the gradients, in the same order as params().
        """
        return ()

In [5]:
class Sigmoid(Layer):
    def forward(self, input: Tensor) -> Tensor:
        """
        Apply sigmoid to each element of the input tensor,
        and save the results to use in backpropagation.
        """
        self.sigmoids = tensor_apply(sigmoid, input)
        return self.sigmoids

    def backward(self, gradient: Tensor) -> Tensor:
        return tensor_combine(lambda sig, grad: sig * (1 - sig) * grad,
                              self.sigmoids,
                              gradient)

In [6]:

def random_uniform(*dims: int) -> Tensor:
    if len(dims) == 1:
        return [random.random() for _ in range(dims[0])]
    else:
        return [random_uniform(*dims[1:]) for _ in range(dims[0])]

def random_normal(*dims: int,
                  mean: float = 0.0,
                  variance: float = 1.0) -> Tensor:
    if len(dims) == 1:
        return [mean + variance * inverse_normal_cdf(random.random())
                for _ in range(dims[0])]
    else:
        return [random_normal(*dims[1:], mean=mean, variance=variance)
                for _ in range(dims[0])]
    
def random_tensor(*dims: int, init: str = 'normal') -> Tensor:
    if init == 'normal':
        return random_normal(*dims)
    elif init == 'uniform':
        return random_uniform(*dims)
    elif init == 'xavier':
        variance = len(dims) / sum(dims)
        return random_normal(*dims, variance=variance)
    else:
        raise ValueError(f"unknown init: {init}")


In [7]:

class Linear(Layer): 
  def __init__(self,
                 input_dim: int,
                 output_dim: int,
                 init: str = 'xavier') -> None:
        """
        A layer of output_dim neurons, each with input_dim weights
        (and a bias).
        """
        self.input_dim = input_dim
        self.output_dim = output_dim

        # self.w[o] is the weights for the oth neuron
        self.w = random_tensor(output_dim, input_dim, init=init)

        # self.b[o] is the bias term for the oth neuron
        self.b = random_tensor(output_dim, init=init)
        
  def forward(self, input: Tensor) -> Tensor:
    # Save the input to use in the backward pass.
    self.input = input

    # Return the vector of neuron outputs.
    return [dot(input, self.w[o]) + self.b[o] for o in range(self.output_dim)]
    
  def backward(self, gradient: Tensor) -> Tensor:
       # Each b[o] gets added to output[o], which means
       # the gradient of b is the same as the output gradient. 
            self.b_grad = gradient

            # Each w[o][i] multiplies input[i] and gets added to output[o].
            # So its gradient is input[i] * gradient[o].
            self.w_grad = [[self.input[i] * gradient[o]
                            for i in range(self.input_dim)]
                           for o in range(self.output_dim)]

            # Each input[i] multiplies every w[o][i] and gets added to every
            # output[o]. So its gradient is the sum of w[o][i] * gradient[o]
            # across all the outputs.
            return [sum(self.w[o][i] * gradient[o] for o in range(self.output_dim))
                    for i in range(self.input_dim)]
  def params(self) -> Iterable[Tensor]:
    return [self.w, self.b]

  def grads(self) -> Iterable[Tensor]:
    return [self.w_grad, self.b_grad]

In [8]:
# def forward(self, input: Tensor) -> Tensor:
#     # Save the input to use in the backward pass.
#     self.input = input

#     # Return the vector of neuron outputs.
#     return [dot(input, self.w[o]) + self.b[o] for o in range(self.output_dim)]

In [9]:
def backward(self, gradient: Tensor) -> Tensor:
   # Each b[o] gets added to output[o], which means
   # the gradient of b is the same as the output gradient. 
        self.b_grad = gradient

        # Each w[o][i] multiplies input[i] and gets added to output[o].
        # So its gradient is input[i] * gradient[o].
        self.w_grad = [[self.input[i] * gradient[o]
                        for i in range(self.input_dim)]
                       for o in range(self.output_dim)]

        # Each input[i] multiplies every w[o][i] and gets added to every
        # output[o]. So its gradient is the sum of w[o][i] * gradient[o]
        # across all the outputs.
        return [sum(self.w[o][i] * gradient[o] for o in range(self.output_dim))
                for i in range(self.input_dim)]

In [10]:
# def params(self) -> Iterable[Tensor]:
#     return [self.w, self.b]

# def grads(self) -> Iterable[Tensor]:
#     return [self.w_grad, self.b_grad]

In [11]:
class Sequential(Layer):
    """
    A layer consisting of a sequence of other layers.
    It's up to you to make sure that the output of each layer
    makes sense as the input to the next layer.
    """
    def __init__(self, layers: List[Layer]) -> None:
        self.layers = layers

    def forward(self, input):
        """Just forward the input through the layers in order."""
        for layer in self.layers:
            input = layer.forward(input)
        return input

    def backward(self, gradient):
        """Just backpropagate the gradient through the layers in reverse."""
        for layer in reversed(self.layers):
            gradient = layer.backward(gradient)
        return gradient

    def params(self) -> Iterable[Tensor]:
        """Just return the params from each layer."""
        return (param for layer in self.layers for param in layer.params())

    def grads(self) -> Iterable[Tensor]:
        """Just return the grads from each layer."""
        return (grad for layer in self.layers for grad in layer.grads())

In [12]:
class Loss:
    def loss(self, predicted: Tensor, actual: Tensor) -> float:
        """How good are our predictions? (Larger numbers are worse.)"""
        raise NotImplementedError

    def gradient(self, predicted: Tensor, actual: Tensor) -> Tensor:
        """How does the loss change as the predictions change?"""
        raise NotImplementedError

In [13]:
class SSE(Loss):
    """Loss function that computes the sum of the squared errors."""
    def loss(self, predicted: Tensor, actual: Tensor) -> float:
        # Compute the tensor of squared differences
        squared_errors = tensor_combine(
            lambda predicted, actual: (predicted - actual) ** 2,
            predicted,
            actual)

        # And just add them up
        return tensor_sum(squared_errors)

    def gradient(self, predicted: Tensor, actual: Tensor) -> Tensor:
        return tensor_combine(
            lambda predicted, actual: 2 * (predicted - actual),
            predicted,
            actual)

In [14]:
class Optimizer:
    """
    An optimizer updates the weights of a layer (in place) using
information
    known by either the layer or the optimizer (or by both).
    """
    def step(self, layer: Layer) -> None:
        raise NotImplementedError

In [15]:
class GradientDescent(Optimizer):
    def __init__(self, learning_rate: float = 0.1) -> None:
        self.lr = learning_rate

    def step(self, layer: Layer) -> None:
        for param, grad in zip(layer.params(), layer.grads()):
            # Update param using a gradient step
            param[:] = tensor_combine(
                lambda param, grad: param - grad * self.lr,
                param,
                grad)

In [16]:
class Momentum(Optimizer):
    def __init__(self,
                 learning_rate: float,
                 momentum: float = 0.9) -> None:
        self.lr = learning_rate
        self.mo = momentum
        self.updates: List[Tensor] = []  # running average

    def step(self, layer: Layer) -> None:
        # If we have no previous updates, start with all zeros
        if not self.updates:
            self.updates = [zeros_like(grad) for grad in layer.grads()]

        for update, param, grad in zip(self.updates,layer.params(), layer.grads()):
            # Apply momentum 
            update[:] = tensor_combine(
                lambda u, g: self.mo * u + (1 - self.mo) * g,
                update,
                grad) 
            # Then take a gradient step
            param[:] = tensor_combine(
                lambda p, u: p - self.lr * u,
                param,
                update)

In [17]:
def softmax(tensor: Tensor) -> Tensor:
    """Softmax along the last dimension"""
    if is_1d(tensor):
        # Subtract largest value for numerical stability.
        largest = max(tensor)
        exps = [math.exp(x - largest) for x in tensor]

        sum_of_exps = sum(exps)                 # This is the total "weight."
        return [exp_i / sum_of_exps             # Probability is the fraction
                for exp_i in exps]              # of the total weight.
    else:
        return [softmax(tensor_i) for tensor_i in tensor]

In [18]:
class SoftmaxCrossEntropy(Loss):
    """
    This is the negative-log-likelihood of the observed values, given the
    neural net model. So if we choose weights to minimize it, our model 
will
    be maximizing the likelihood of the observed data.
    """
    def loss(self, predicted: Tensor, actual: Tensor) -> float:
        # Apply softmax to get probabilities
        probabilities = softmax(predicted)

        # This will be log p_i for the actual class i and 0 for the other
        # classes. We add a tiny amount to p to avoid taking log(0).
        likelihoods = tensor_combine(lambda p, act: math.log(p + 1e-30) * act,
                                     probabilities,
                                     actual)

        # And then we just sum up the negatives.
        return -tensor_sum(likelihoods)

    def gradient(self, predicted: Tensor, actual: Tensor) -> Tensor:
        probabilities = softmax(predicted)

        # Isn't this a pleasant equation?
        return tensor_combine(lambda p, actual: p - actual,
                              probabilities,
                              actual)

Example: XOR Revisited

In [19]:
# training data

# X=[[1,0,1,0],[1,0,1,1],[0,1,0,1]]
# # creating the output array
# y= [[1],[1],[0]]
# y1= [[1,0,1],[0,1,0],[1,0,0]]

# xs = np.array(X)
# ys = np.array(y)

# xs = [[0., 0,1], [0., 1,1], [1., 0,0], [1., 1,0]]
# ys = [[0.], [1.], [1.], [0.]]

# xs = [10, 20, 5, 40]

# friend_counts = Counter(num_friends)

# x2 = range(101)                     
# y2 = [friend_counts[x] for x in x2] 

# x2 = [np.random.normal() for _ in range(1000)]
# y2 = [ x + np.random.normal() / 2 for x in x2]

X1 = np.random.randint(5,size=[3,4])
y1 = np.random.randint(5,size=[3,1])

xs = np.array(X1)
ys = np.array(y1)
ys2 = ys
ys2



array([[4],
       [2],
       [0]])

In [20]:
# for x, y in zip(xs, ys):
#     abc = x
# #     print(x)
# print(abc)
# predicted = net.forward(abc)
# # predicted

In [21]:
random.seed(0)

net = Sequential([Linear(input_dim=4, output_dim=3), Sigmoid(), Linear(input_dim=3, output_dim=1)])
# net.backward()

In [22]:
for x, y in zip(xs, ys):
    abc = x
#     print(x)
print(abc)
abc
predicted = net.forward(abc)
predicted
# abc
# type(abc.tolist())

[4 3 2 4]


[-0.20042415388425328]

In [23]:

def tanh(x: float) -> float:
    # If x is very large or very small, tanh is (essentially) 1 or -1.
    # We check for this because, e.g., math.exp(1000) raises an error.
    if x < -100:  return -1
    elif x > 100: return 1

    em2x = math.exp(-2 * x)
    return (1 - em2x) / (1 + em2x)


class Tanh(Layer):
    def forward(self, input: Tensor) -> Tensor:
        # Save tanh output to use in backward pass.
        self.tanh = tensor_apply(tanh, input)
        return self.tanh

    def backward(self, gradient: Tensor) -> Tensor:
        return tensor_combine(
            lambda tanh, grad: (1 - tanh ** 2) * grad,
            self.tanh,
            gradient)

In [24]:
# optimizer = GradientDescent(learning_rate=0.1)
# loss = SSE()


optimizer = Momentum(learning_rate=0.1, momentum=0.9)
loss = SoftmaxCrossEntropy()


# predicted = net.forward(xs[0])

pred = []
epochloss = []
gradi = []
loss_loss = []

with tqdm.trange(3000) as t: 
    for epoch in t:
        epoch_loss = 0.0

        for x, y in zip(xs, ys):
            abc = x
            print(abc)
            predicted = net.forward(abc)
            pred.append(predicted)
            loss_loss.append(loss.loss(predicted, y))
            epoch_loss += loss.loss(predicted, y)
            epochloss.append(epoch_loss)
            gradient = loss.gradient(predicted, y)
            gradi.append(gradient)
            net.backward(gradient)

            optimizer.step(net)
            
#         pred.append(predicted)

        t.set_description(f"xor loss {epoch_loss:.3f}")

  0%|                                                                                                                                             | 0/3000 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp/ipykernel_40532/1617746550.py:40: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return [sum(self.w[o][i] * gradient[o] for o in range(self.output_dim))
xor loss 0.000:   1%|█▎                                                                                                                 | 33/3000 [00:00<00:09, 325.30it/s]

[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]


xor loss 0.000:   1%|█▎                                                                                                                 | 33/3000 [00:00<00:09, 325.30it/s]

[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]


xor loss 0.000:   3%|███▉                                                                                                              | 104/3000 [00:00<00:08, 336.97it/s]

...

[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]


xor loss 0.000:  98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 2947/3000 [00:11<00:00, 279.42it/s]


[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]

xor loss 0.000: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:11<00:00, 267.48it/s]


[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]
[2 3 3 3]
[4 3 2 4]
[4 3 4 3]

In [25]:
# for x, y in zip(xs, ys):
#     abc = x
#     print(abc)
# predicted = net.forward(abc)

In [26]:
# loss_loss
np.array(pred).reshape(3000,3)
abc

array([4, 3, 2, 4])

In [27]:
param0 = []
for param in net.params():
    param0.append(param)
# param0

In [28]:
# pred

In [29]:
# hidden1 = -2.6 * x1 + -2.7 * x2 + 0.2  # NOR
# hidden2 =  2.1 * x1 +  2.1 * x2 - 3.4  # AND
# output =  -3.1 * h1 + -2.6 * h2 + 1.8  # NOR

In [30]:

def tanh(x: float) -> float:
    # If x is very large or very small, tanh is (essentially) 1 or -1.
    # We check for this because, e.g., math.exp(1000) raises an error.
    if x < -100:  return -1
    elif x > 100: return 1

    em2x = math.exp(-2 * x)
    return (1 - em2x) / (1 + em2x)


class Tanh(Layer):
    def forward(self, input: Tensor) -> Tensor:
        # Save tanh output to use in backward pass.
        self.tanh = tensor_apply(tanh, input)
        return self.tanh

    def backward(self, gradient: Tensor) -> Tensor:
        return tensor_combine(
            lambda tanh, grad: (1 - tanh ** 2) * grad,
            self.tanh,
            gradient)

In [31]:
class Relu(Layer):
    def forward(self, input: Tensor) -> Tensor:
        self.input = input
        return tensor_apply(lambda x: max(x, 0), input)
 
    def backward(self, gradient: Tensor) -> Tensor:
        return tensor_combine(lambda x, grad: grad if x > 0 else 0,
                              self.input,
                              gradient)

In [32]:
param

[899.9921809362888]

In [33]:
net.params()

<generator object Sequential.params.<locals>.<genexpr> at 0x000001E5EE6C6890>

In [34]:
aqd = np.array(pred).reshape(3000,3)
aqd[-1]
error1 = []
for i in range(len(aqd)):
    error1.append(np.mean(aqd[i]-ys.T))

# min_ind = error1.index((min(np.abs(error1))))
# print(aqd[min_ind])
# print(min(error1))
# aqd[-1]-ys.T
# print(aqd[-1])
# print(error1)
# min_ind
aqd[-1]

array([2624.46860852, 2624.79013412, 2625.11051052])

In [35]:
ys2

array([[4],
       [2],
       [0]])

In [36]:

# print(np.sum(aqd[-1]))
for i in range(len(aqd)):
    q = np.sum(aqd[i])
    for j in range(len(aqd[i])):
        aqd[i][j] = aqd[i][j]/q

#     for j in range(len(aqd[i])):
#         aqd[j] = aqd[j]/q
#         print(aqd[j])

# q = np.sum(aqd[-1])
# aqd[-1][0]/q
# aqd
# print(aqd[-1])

In [37]:
# epochloss

In [38]:
# gradi